# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 12 2022 2:41PM,244103,10,SOTAH0000753,Ivaoes Animal Health,Released
1,Aug 12 2022 2:41PM,244102,10,SONSB0001895,"Nextsource Biotechnology, LLC",Released
2,Aug 12 2022 2:35PM,244100,20,8394005,Sartorius Stedim North America,Released
3,Aug 12 2022 2:16PM,244096,10,0085937381,ISDIN Corporation,Executing
4,Aug 12 2022 2:16PM,244096,10,0085937384,ISDIN Corporation,Executing
5,Aug 12 2022 2:16PM,244096,10,0085937385,ISDIN Corporation,Executing
6,Aug 12 2022 2:16PM,244096,10,0085937387,ISDIN Corporation,Executing
7,Aug 12 2022 2:16PM,244096,10,0085937388,ISDIN Corporation,Executing
8,Aug 12 2022 2:16PM,244096,10,0085937389,ISDIN Corporation,Executing
9,Aug 12 2022 2:16PM,244096,10,0085937386,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,244098,Executing,4
38,244098,Released,3
39,244100,Released,1
40,244102,Released,1
41,244103,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244097,NaN,NaN,1.0
244098,NaN,4.0,3.0
244100,NaN,NaN,1.0
244102,NaN,NaN,1.0
244103,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244001,0.0,0.0,1.0
244014,0.0,0.0,1.0
244023,0.0,0.0,12.0
244024,3.0,2.0,19.0
244025,0.0,1.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244001,0,0,1
244014,0,0,1
244023,0,0,12
244024,3,2,19
244025,0,1,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244001,0,0,1
1,244014,0,0,1
2,244023,0,0,12
3,244024,3,2,19
4,244025,0,1,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244001,,,1
1,244014,,,1
2,244023,,,12
3,244024,3,2,19
4,244025,,1,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 12 2022 2:41PM,244103,10,Ivaoes Animal Health
1,Aug 12 2022 2:41PM,244102,10,"Nextsource Biotechnology, LLC"
2,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America
3,Aug 12 2022 2:16PM,244096,10,ISDIN Corporation
42,Aug 12 2022 2:14PM,244098,16,Sartorius Bioprocess Solutions
49,Aug 12 2022 2:11PM,244097,19,ACG North America LLC
50,Aug 12 2022 2:07PM,244094,10,Acute Outpatient Solutions
51,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc."
52,Aug 12 2022 2:05PM,244091,10,ISDIN Corporation
60,Aug 12 2022 2:05PM,244092,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 12 2022 2:41PM,244103,10,Ivaoes Animal Health,,,1
1,Aug 12 2022 2:41PM,244102,10,"Nextsource Biotechnology, LLC",,,1
2,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,,,1
3,Aug 12 2022 2:16PM,244096,10,ISDIN Corporation,,16,23
4,Aug 12 2022 2:14PM,244098,16,Sartorius Bioprocess Solutions,,4,3
5,Aug 12 2022 2:11PM,244097,19,ACG North America LLC,,,1
6,Aug 12 2022 2:07PM,244094,10,Acute Outpatient Solutions,,,1
7,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",,,1
8,Aug 12 2022 2:05PM,244091,10,ISDIN Corporation,,2,6
9,Aug 12 2022 2:05PM,244092,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 2:41PM,244103,10,Ivaoes Animal Health,1,,
1,Aug 12 2022 2:41PM,244102,10,"Nextsource Biotechnology, LLC",1,,
2,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1,,
3,Aug 12 2022 2:16PM,244096,10,ISDIN Corporation,23,16,
4,Aug 12 2022 2:14PM,244098,16,Sartorius Bioprocess Solutions,3,4,
5,Aug 12 2022 2:11PM,244097,19,ACG North America LLC,1,,
6,Aug 12 2022 2:07PM,244094,10,Acute Outpatient Solutions,1,,
7,Aug 12 2022 2:05PM,244093,21,"NBTY Global, Inc.",1,,
8,Aug 12 2022 2:05PM,244091,10,ISDIN Corporation,6,2,
9,Aug 12 2022 2:05PM,244092,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 2:41PM,244103,10,Ivaoes Animal Health,1,,
1,Aug 12 2022 2:41PM,244102,10,"Nextsource Biotechnology, LLC",1,,
2,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1,,
3,Aug 12 2022 2:16PM,244096,10,ISDIN Corporation,23,16,
4,Aug 12 2022 2:14PM,244098,16,Sartorius Bioprocess Solutions,3,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 2:41PM,244103,10,Ivaoes Animal Health,1.0,NaN,NaN
1,Aug 12 2022 2:41PM,244102,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1.0,NaN,NaN
3,Aug 12 2022 2:16PM,244096,10,ISDIN Corporation,23.0,16.0,NaN
4,Aug 12 2022 2:14PM,244098,16,Sartorius Bioprocess Solutions,3.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 12 2022 2:41PM,244103,10,Ivaoes Animal Health,1.0,0.0,0.0
1,Aug 12 2022 2:41PM,244102,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Aug 12 2022 2:35PM,244100,20,Sartorius Stedim North America,1.0,0.0,0.0
3,Aug 12 2022 2:16PM,244096,10,ISDIN Corporation,23.0,16.0,0.0
4,Aug 12 2022 2:14PM,244098,16,Sartorius Bioprocess Solutions,3.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3172974,99.0,21.0,3.0
102,244014,1.0,0.0,0.0
12,244062,0.0,1.0,0.0
15,244036,1.0,0.0,0.0
16,1952432,8.0,6.0,0.0
19,244097,1.0,0.0,0.0
20,1220334,68.0,27.0,7.0
21,732275,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3172974,99.0,21.0,3.0
1,102,244014,1.0,0.0,0.0
2,12,244062,0.0,1.0,0.0
3,15,244036,1.0,0.0,0.0
4,16,1952432,8.0,6.0,0.0
5,19,244097,1.0,0.0,0.0
6,20,1220334,68.0,27.0,7.0
7,21,732275,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,99.0,21.0,3.0
1,102,1.0,0.0,0.0
2,12,0.0,1.0,0.0
3,15,1.0,0.0,0.0
4,16,8.0,6.0,0.0
5,19,1.0,0.0,0.0
6,20,68.0,27.0,7.0
7,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,99.0
1,102,Released,1.0
2,12,Released,0.0
3,15,Released,1.0
4,16,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
Executing,21.0,0.0,1.0,0.0,6.0,0.0,27.0,0.0
Released,99.0,1.0,0.0,1.0,8.0,1.0,68.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
1,Executing,21.0,0.0,1.0,0.0,6.0,0.0,27.0,0.0
2,Released,99.0,1.0,0.0,1.0,8.0,1.0,68.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
1,Executing,21.0,0.0,1.0,0.0,6.0,0.0,27.0,0.0
2,Released,99.0,1.0,0.0,1.0,8.0,1.0,68.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()